In [1]:
import sqlite3
import json
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import SQLDatabase
from dotenv import load_dotenv, find_dotenv
import sys
import stat
from langchain_community.llms import HuggingFaceEndpoint
from langchain_anthropic import ChatAnthropic

c:\Users\tomce\miniconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Load up API Keys

In [2]:
load_dotenv(find_dotenv())
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

#### Create Database File for Mobile Phones

In [164]:
# Function to create database and tables
def create_database():
    conn = sqlite3.connect('product_database12.db')
    c = conn.cursor()

    # Create products table
    c.execute('''CREATE TABLE IF NOT EXISTS mobile_phones (
                    id INTEGER PRIMARY KEY,
                    url TEXT,
                    title TEXT,
                    warranty INTEGER,
                    regular_price REAL,
                    happy_price REAL,
                    category TEXT
                 )''')

    # Create feature_products table
    c.execute('''CREATE TABLE IF NOT EXISTS feature_mobile_phones (
                    id INTEGER PRIMARY KEY,
                    feature_name TEXT,
                    feature_value TEXT
                 )''')

    # Create product_feature table
    c.execute('''CREATE TABLE IF NOT EXISTS mobile_phone_feature (
                    product_id INTEGER,
                    feature_id INTEGER,
                    FOREIGN KEY (product_id) REFERENCES mobile_phones(id),
                    FOREIGN KEY (feature_id) REFERENCES feature_mobile_phones(id)
                 )''')

    conn.commit()
    conn.close()

# Function to insert product data into database
def insert_product_data(product_data):
    conn = sqlite3.connect('product_database12.db')
    c = conn.cursor()

    # Insert product data into products table
    c.execute('''INSERT INTO mobile_phones (url, title, warranty, regular_price, happy_price, category)
                 VALUES (?, ?, ?, ?, ?, ?)''', (
                    product_data.get('url'),
                    product_data.get('title'),
                    product_data.get('warranty_months'),
                    product_data.get('regular_price_mkd'),
                    product_data.get('happy_price_mkd'),
                    product_data.get('category')
                 ))
    product_id = c.lastrowid

    # Insert feature data into feature_products table and product_feature table
    for feature, value in product_data.get('features', {}).items():
        # Check if feature already exists
        c.execute('''SELECT id FROM feature_mobile_phones WHERE feature_name = ? AND feature_value = ?''', (feature, value))
        feature_row = c.fetchone()
        if feature_row:
            feature_id = feature_row[0]
        else:
            c.execute('''INSERT INTO feature_mobile_phones (feature_name, feature_value) VALUES (?, ?)''', (feature, value))
            feature_id = c.lastrowid

        # Insert into product_feature table
        c.execute('''INSERT INTO mobile_phone_feature (product_id, feature_id) VALUES (?, ?)''',
                  (product_id, feature_id))

    conn.commit()
    conn.close()

# Function to read JSON files from a directory and insert data into database
def process_json_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            with open(os.path.join(directory, filename), 'r') as file:
                product_data = json.load(file)
                insert_product_data(product_data)

# Create database and tables
create_database()

# Process JSON files and insert data into database
json_directory = r'C:\Users\tomce\OneDrive - UKIM, FINKI\Desktop\Fakultet 3ta Godina\2 Sesti Semestar\0 DATA SCIENCE SEMINARSKA\1 Starting Over\products_categories_fixed\mobile_phones\mobile_phones_new_template'
process_json_files(json_directory)


#### Get All "feature_name=feature_value" pairs for each key-value 

In [ ]:
def get_features_string(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Query to select all features
    cursor.execute("SELECT feature_name, feature_value FROM feature_mobile_phones WHERE feature_value IS NOT NULL")
    
    # Fetch all rows from the executed query
    features = cursor.fetchall()
    
    # Close the database connection
    conn.close()
    
    # Format the features into the desired string format
    features_string = ", ".join([f"{name}={value}" for name, value in features])
    
    return features_string

# Example usage
db_path = 'product_database11.db'
features_string = get_features_string(db_path)
print(features_string)

#### Connect with Database File

In [165]:
conn = sqlite3.connect('product_database12.db')
c = conn.cursor()
input_db = SQLDatabase.from_uri('sqlite:///product_database11.db')

#### Loading Features Table From .txt File (Not Needed Anymore)

In [12]:
pat_dir = r"C:\Users\tomce\OneDrive - UKIM, FINKI\Desktop\Fakultet 3ta Godina\2 Sesti Semestar\0 DATA SCIENCE SEMINARSKA\1 Starting Over\products_categories_fixed\mobile_phones\templates\feature2.txt"

os.chmod(pat_dir, stat.S_IREAD | stat.S_IWRITE)

with open(pat_dir, "r") as f: features = f.read()

#### Chat Prompt Template for Communicating With Mobile Phones

In [166]:
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Please note that in the schema, the regular_price is the normal price, while the happy_price is a discounted one.
Table Descriptions:
mobile_phones Table:
This table stores general information about mobile phones.
Columns:
id: Unique identifier for each mobile phone.
url: URL link to the product page.
title: Title or name of the mobile phone.
warranty: Duration of warranty in months.
regular_price: Normal price of the mobile phone.
happy_price: Discounted price (if applicable).
category: Category of the mobile phone (e.g., “gaming_laptops”).

feature_mobile_phones Table:
This table captures individual features of mobile phones.
Columns:
id: Unique identifier for each feature.
feature_name: Name of the feature (e.g., “camera resolution,” “battery capacity”).
feature_value: Value associated with the feature (e.g., “12 MP,” “4000 mAh”).

mobile_phone_feature Table:
This table establishes a many-to-many relationship between mobile phones and their features.
Columns:
product_id: Foreign key referencing the id column in the mobile_phones table.
feature_id: Foreign key referencing the id column in the feature_mobile_phones table.

Example input for questions asking for multiple features: 'What phones are there that have a screen size of at least 6 inches and an OLED screen?'
Example output for questions asking for multiple features:
'SELECT mobile_phones.title, screen_size.feature_value AS screen_size, screen_type.feature_value AS screen_type
FROM mobile_phones
JOIN mobile_phone_feature AS mpf1 ON mobile_phones.id = mpf1.product_id
JOIN feature_mobile_phones AS screen_size ON mpf1.feature_id = screen_size.id
JOIN mobile_phone_feature AS mpf2 ON mobile_phones.id = mpf2.product_id
JOIN feature_mobile_phones AS screen_type ON mpf2.feature_id = screen_type.id
WHERE screen_size.feature_name = 'screen_size_inches'
AND CAST(screen_size.feature_value AS UNSIGNED) >= 6
AND screen_type.feature_name = 'screen_type'
AND screen_type.feature_value LIKE '%OLED%';'
Don't include a feature in the query if the user doesn't ask for it.

You ABSOLUTELY MUST use the ‘LIKE’ operator instead of ‘=’ in the SQL query for columns in the tables that are of TEXT value.
You ABSOLUTELY MUST use the ‘=’ operator instead of ‘LIKE’ in the SQL query for columns in the tables that are of REAL, DOUBLE, or INTEGER value.
Here are all the features a user might ask for:
{features}
Question: {question}
SQL Query:
"""
prompt = ChatPromptTemplate.from_template(template)

def get_schema(db):
    schema = input_db.get_table_info()
    return schema

#### 3 Models to Choose From: HuggingFace Mistral, Gemini, and Claude Sonnet

In [157]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=128, temperature=0.1, token=HUGGINGFACEHUB_API_TOKEN
)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\tomce\.cache\huggingface\token
Login successful


In [150]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.1)

In [167]:
llm = ChatAnthropic(temperature=0, model_name="claude-3-sonnet-20240229")

#### Ask Question to Get Query Answer

In [ ]:
sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

user_question = 'Can you give me all the phones with a camera that is at least 50MP and the size is at least 6 inches?'
smth= sql_chain.invoke({"features": features_string,"question": user_question})
print(smth)
output_string = smth.replace("```sql", "").replace("```", "")
print(output_string)

#### BELOW: CLAUDE, GOOD QUERY

In [163]:
sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

user_question = 'Can you give me all the phones with a camera that is at least 50MP and the size is at least 6 inches?'
smth= sql_chain.invoke({"features": features_string,"question": user_question})
print(smth)
output_string = smth.replace("```sql", "").replace("```", "")
print(output_string)

Here's the SQL query to find all phones with a camera that is at least 50MP and a screen size of at least 6 inches:

```sql
SELECT mobile_phones.title
FROM mobile_phones
JOIN mobile_phone_feature mpf1 ON mobile_phones.id = mpf1.product_id
JOIN feature_mobile_phones screen_size ON mpf1.feature_id = screen_size.id
JOIN mobile_phone_feature mpf2 ON mobile_phones.id = mpf2.product_id
JOIN feature_mobile_phones camera_mp ON mpf2.feature_id = camera_mp.id
WHERE screen_size.feature_name = 'screen_size_inches'
  AND CAST(screen_size.feature_value AS UNSIGNED) >= 6
  AND camera_mp.feature_name LIKE '%primary_wide_camera_mp%'
  AND CAST(camera_mp.feature_value AS UNSIGNED) >= 50;
```

Explanation:

1. We join the `mobile_phones` table with the `mobile_phone_feature` and `feature_mobile_phones` tables to access the screen size and camera megapixel information.
2. The `WHERE` clause filters the results based on the following conditions:
   - `screen_size.feature_name = 'screen_size_inches'` and `C

#### BELOW CLAUDE, GOOD OUTPUT EXCEPT FOR SELECT WITH ERROR THAT TELLS THESE DON'T EXIST : "feature_mobile_phones.feature_value AS screen_size, feature_mobile_phones.feature_value AS screen_type"

In [168]:
sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

user_question = 'Can you give me all the phones with a camera that has an OLED screen and is at least 6 inches and the price is under 60000?'
smth= sql_chain.invoke({"features": features_string,"question": user_question})
print(smth)

SELECT mobile_phones.title, feature_mobile_phones.feature_value AS screen_size, feature_mobile_phones.feature_value AS screen_type
FROM mobile_phones
JOIN mobile_phone_feature AS mpf1 ON mobile_phones.id = mpf1.product_id
JOIN feature_mobile_phones AS screen_size ON mpf1.feature_id = screen_size.id
JOIN mobile_phone_feature AS mpf2 ON mobile_phones.id = mpf2.product_id
JOIN feature_mobile_phones AS screen_type ON mpf2.feature_id = screen_type.id
WHERE screen_size.feature_name = 'screen_size_inches'
AND CAST(screen_size.feature_value AS UNSIGNED) >= 6
AND screen_type.feature_name = 'screen_type'
AND screen_type.feature_value LIKE '%OLED%'
AND mobile_phones.happy_price < 60000;
